In [6]:
# !pip install -r requirements.txt 

In [ ]:
import cv2 
import torch
import numpy as np 

# Load the MiDaS model
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_large")
midas.eval()

# Load the MiDaS transforms
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms").small_transform 

# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas.to(device)

# Estimating Depth Fn
def estimate_depth(frame):
    input_batch = midas_transforms(frame).to(device)
    with torch.no_grad():
        prediction = midas(input_batch)
        depth_map = prediction.squeeze().cpu().numpy()
        return depth_map 
    
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break 

    # Resize the input image to match the model input 
    input_frame = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), (256,256))

    # Estimate depth
    depth_map = estimate_depth(input_frame)

    # Normalize the depth map for visualization
    depth_map_normalize = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    depth_map_colored = cv2.applyColorMap(depth_map_normalize, cv2.COLORMAP_JET)

    # Display both the original frame and depth map
    combined = np.hstack((frame, cv2.resize(depth_map_colored,(frame.shape[1], frame.shape[0]))))
    cv2.imshow("Depth Detection", combined)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


